# Load Data

In [34]:
hyperparameters = {'hidden_dim': 64, 'batch_size': 128, 'n1_gat_layers': 1, 'n2_gru_layers': 1, 'num_heads': 4, 'dropout': 0.04033931265087129, 'learning_rate': 0.00015574186652855083, 'pos_lambda': 0.5756532880616873, 'bios_hidden_dim': 32, 'pres_hidden_dim': 32, 'k': 5, 'num_clusters': 240}

In [55]:
import preprocess
import imp
imp.reload(preprocess)

data = preprocess.preprocess_pipeline(num_clusters=hyperparameters['num_clusters'])

1. Include only first admissions: N=32513
2. Exclusion by ages: N=25548
3. Include only patients who admitted for at least 24 hours: N=25168
4. Exclude patients who died within 54-hours of admission: N=24556


/home/bnet/kupershmidt/HML/preprocess.py:209: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pivot = pd.pivot_table(merged, index=['subject_id', 'hadm_id', pd.Grouper(key='charttime', freq=PREDICT_FREQ)],
/home/bnet/kupershmidt/HML/preprocess.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=["mort_30day","prolonged_stay","readmission_30day"], axis=1, inplace=True)
/home/bnet/kupershmidt/HML/preprocess.py:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val.drop(columns=["mort_30day","prolonged_stay","readmission_30day"], axis=1, inplace=

Pooled embeddings shape: 24529
Number of subjects: 24529


# Datasets creation

In [56]:
data["prescriptions_train"].shape[0] == data["padded_tensor_train"].shape[0]

True

In [57]:
import Dataset
import imp
imp.reload(Dataset)
import torch
from torch.utils.data import DataLoader
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k = hyperparameters['k']

train_labels = torch.tensor(data["y_train"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)
val_labels = torch.tensor(data["y_val"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)
test_labels = torch.tensor(data["y_test"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)


batch_size = hyperparameters['batch_size']
datasets = {x: Dataset.PatientDataset(d, y, core=data["padded_tensor_core"], padding_mask=m, padding_mask_core=data["padding_mask_core"], k=k ,notes=n, bios=b, prescriptions=p) for x, d, y, m, n, b, p in
        zip(['train', 'val', 'test'], [data["padded_tensor_train"], data["padded_tensor_val"], data["padded_tensor_test"]],
            [train_labels, val_labels, test_labels],
            [data["padding_mask_train"], data["padding_mask_val"], data["padding_mask_test"]],
            [torch.stack(data["notes_df_train"].embeddings.values.tolist()),
             torch.stack(data["notes_df_val"].embeddings.values.tolist()),
             torch.stack(data["notes_df_test"].embeddings.values.tolist())],
             [torch.tensor(data["bio_train"].values >= 1, dtype=torch.float32).to(DEVICE),
              torch.tensor(data["bio_val"].values >= 1, dtype=torch.float32).to(DEVICE),
              torch.tensor(data["bio_test"].values >= 1, dtype=torch.float32).to(DEVICE)],
              [torch.tensor(data["prescriptions_train"].values, dtype=torch.float32).to(DEVICE), 
              torch.tensor(data["prescriptions_val"].values, dtype=torch.float32).to(DEVICE), 
              torch.tensor(data["prescriptions_test"].values, dtype=torch.float32).to(DEVICE)])}
dataloaders = {x: DataLoader(datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val', 'test']}

# Model Training

In [60]:
import Model 
import imp
imp.reload(Model)
model = Model.GraphGRUMortalityModel(input_dim=data["padded_tensor_train"].shape[2], hidden_dim=hyperparameters['hidden_dim'], 
                                     n1_gat_layers=2, n2_gru_layers=1, 
                                     X_core=data["padded_tensor_core"], num_of_bios=data["bio_train"].shape[1],
                                     num_prescriptions=data["prescriptions_train"].shape[-1], 
                                     bios_hidden_dim=hyperparameters['bios_hidden_dim'], pres_hidden_dim=hyperparameters['pres_hidden_dim'], 
                                     core_padding_mask=data["padding_mask_core"], num_heads=hyperparameters['num_heads'], 
                                     dropout=hyperparameters['dropout'], seq_len=data["padded_tensor_train"].shape[1], k=k, gnn_flag=False).to(DEVICE)
torch.manual_seed(1234)
model.train_all(dataloaders, datasets, epochs=10, learning_rate=hyperparameters['learning_rate'], pos_lambda=hyperparameters['pos_lambda'])
print("Training completed. Validating on test set...")

model.validate(dataloaders['test'], datasets['test'])
#print(f'Test Accuracy in model: {acc:.4f}, AUC: {auc:.4f}, AP: {pr:.4f}')

Pos weight 0: 7.5538
Pos weight 1: 1.1253
Pos weight 2: 23.8500
Starting epoch 1/10
100%|██████████| 152/152 [01:29<00:00,  1.70it/s]
Epoch 1/10, Loss: 2.3834
100%|██████████| 20/20 [00:11<00:00,  1.73it/s]
Val Mortality - AUC: 0.7387 | AP: 0.2987
Val Prolonged LOS - AUC: 0.7005 | AP: 0.6752
Val Readmission - AUC: 0.5604 | AP: 0.0474
Best model updated
Starting epoch 2/10
100%|██████████| 152/152 [01:34<00:00,  1.62it/s]
Epoch 2/10, Loss: 2.2576
100%|██████████| 20/20 [00:11<00:00,  1.78it/s]
Val Mortality - AUC: 0.7743 | AP: 0.3333
Val Prolonged LOS - AUC: 0.7356 | AP: 0.7048
Val Readmission - AUC: 0.6125 | AP: 0.0556
Best model updated
Starting epoch 3/10
100%|██████████| 152/152 [01:28<00:00,  1.71it/s]
Epoch 3/10, Loss: 2.2027
100%|██████████| 20/20 [00:11<00:00,  1.77it/s]
Val Mortality - AUC: 0.7897 | AP: 0.3498
Val Prolonged LOS - AUC: 0.7542 | AP: 0.7252
Val Readmission - AUC: 0.6268 | AP: 0.0586
Best model updated
Starting epoch 4/10
100%|██████████| 152/152 [01:29<00:00,  1.6

(0.8342341733666645,
 0.4550947284802778,
 0.7827208694437608,
 0.7601116657029308,
 0.6215462319144759,
 0.08462695438594925)

In [61]:
res = model.validate(dataloaders['test'], datasets['test'])

print(f"Test Mortality: AUC-ROC: {res[0]:.2f}  AP: {res[1]:.2f}")
print(f"Test Prolonged Stay: AUC-ROC: {res[2]:.2f}  AP: {res[3]:.2f}")
print(f"Test Readmission: AUC-ROC: {res[4]:.2f}  AP: {res[5]:.2f}")

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:11<00:00,  1.72it/s]
Test Mortality: AUC-ROC: 0.83  AP: 0.46
Test Prolonged Stay: AUC-ROC: 0.78  AP: 0.76
Test Readmission: AUC-ROC: 0.62  AP: 0.08


In [62]:
torch.save(model, 'baseline_model.pt')


In [ ]:
import pandas as pd
x, y, padding_mask, idx, notes, bios, prescriptions = next(iter(dataloaders['test']))
edge_index = datasets['test'].get_edge_index(x, padding_mask, idx).to(DEVICE)
pd.DataFrame(edge_index.T.cpu().numpy()).to_csv('edge_index.csv', index=False)

<!-- 100%|██████████| 10/10 [00:07<00:00,  1.38it/s]
Test Mortality: AUC-ROC: 0.85  AP: 0.52
Test Prolonged Stay: AUC-ROC: 0.82  AP: 0.80
Test Readmission: AUC-ROC: 0.62  AP: 0.08 -->

<!-- Test Mortality: AUC-ROC: 0.87  AP: 0.52
Test Prolonged Stay: AUC-ROC: 0.83  AP: 0.81
Test Readmission: AUC-ROC: 0.62  AP: 0.08 -->